In [1]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report

In [9]:
train_dir="D:\ML Projects\Blood-Cell-Classification\dataset2-master\images\TRAIN"
test_dir="D:\ML Projects\Blood-Cell-Classification\dataset2-master\images\TEST"

In [5]:
#generators as it prevents loading all the images onto the disk and loads a few of them and recyles the memor
#here we create gneraetoirs


train_gen=tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)
test_gen=tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input, 
)

In [10]:
#flowing Image data here

train_images=train_gen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 244),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,#shuffing image order
    seed=42,
    subset='training'
)
val_images=train_gen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 244),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)
test_images=test_gen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 244),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,#dont want to shuffle and it doesnt matter anyways
    seed=42,
)

Found 7968 images belonging to 4 classes.
Found 1989 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.


In [ ]:
#Build a pretrained Model
pretrained_model=tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,#removes the classification aspect
    weights='imagenet',
    pooling='avg',#reduces the op from 2d to 1d 
)

pretrained_model.trainable = False


In [ ]:
#Classifictiaon Model
